### Selected Word Group In Youtube Sentence

In [20]:
import os
import re
import pandas as pd
import numpy as np
import glob
from functools import reduce
from pathlib import Path
import shutil
from itertools import islice

In [21]:
# language pair
lang_folder = "Turkish"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
lang_pair = "Dutch"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# source path
source_path = "/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Turkish/Deployment2/Result/Deploy2 Result Manuel/Turkish Dutch"

# parameter
file = "Turkish Dutch SV"
sheets = "2 gram hybrid"  # 2 gram target, 2 gram hybrid
start_first_column = 0  # start column location number
time_shift = 0.6
sample_num = 6

In [22]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Deployment2/Result/3-Selected Word Group In Youtube Sentence/{lang_folder.capitalize()} {lang_pair.capitalize()}"

Path(path).mkdir(parents=True, exist_ok=True)

In [23]:
def word_group_youtube(df, search_list, target_column, sample_num):
    '''
    word_group_youtube(df_youtube_sentence, search_list, "sentence", 6)
    ''' 
    df_search_result = pd.DataFrame()
    for j in search_list:
        try:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].sample(sample_num)
        except:
            df_select = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)].head(sample_num)
        #df_result = df[df[f"{target_column}"].str.contains(fr"(?:\s|^){j}(?:\s|$)", na=True)]  # sentence length part
        #df_result.sort_values(f"{target_column}",key=lambda x:x.str.len(), inplace=True)
        #df_select = df_result.head(sample_num)
        df_select.insert(0,"search_string",j)
        df_search_result = pd.concat([df_search_result,df_select], axis=0)
    df_search_result.reset_index(inplace=True, drop=True)
    
    return df_search_result

In [24]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [25]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:02:51.948,00:02:58.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,00:03:00.956,00:03:04.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,00:03:13.434,00:03:16.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,00:03:17.235,00:03:21.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,00:03:27.806,00:03:33.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,00:11:07.990,00:11:14.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,00:11:14.560,00:11:21.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,00:11:21.040,00:11:27.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,00:11:27.880,00:11:32.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [26]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [27]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,171.948,178.829,özgür bunlar normalde kamyon daha büyük araçla...,8V9tq1pe8eI
1,180.956,184.236,burcu arka tarafı bağlamak kolay olmayacak,8V9tq1pe8eI
2,193.434,196.327,özgür arabaya yarım tur attıracağım,8V9tq1pe8eI
3,197.235,201.338,burcu biraz daha devam et devam et tamam oldu,8V9tq1pe8eI
4,207.806,213.383,burcu şimdilik iki tekere takacağız ama kar ka...,8V9tq1pe8eI
...,...,...,...,...
3063204,667.990,674.560,oynatma listesi linkini videonun sağ üst köşes...,MvWp9pWLihA
3063205,674.560,681.040,kısmına ekledim dilediğiniz kenar deseni uyarl...,MvWp9pWLihA
3063206,681.040,687.880,ibaretti bir sonraki farklı bir elle örgü eğit...,MvWp9pWLihA
3063207,687.880,692.290,kanalıma abone olarak videomu beğenmeyi unutma...,MvWp9pWLihA


In [28]:
df_word_group_select = pd.read_excel(f"{source_path}/{file}.xlsx", sheet_name= f"{sheets}")
df_word_group_select

,twogram_pair1
0,ajan rolü
1,alarm şifresi
2,alerji testi
3,alerjik şok
4,alerjik şoka
...,...
351,video konferansı
352,video mesajı
353,video sistemi
354,video ünitesi


In [29]:
search_list = []
for i in range(len(df_word_group_select)):
    for j in df_word_group_select.columns[start_first_column:]:
        string = df_word_group_select.loc[i,j]
        
        if pd.isnull(string) == False and string != 'nan':
            search_list.append(string.strip())
        else:
            pass

In [30]:
df_search_result = word_group_youtube(df_youtube_sentence, search_list, "sentence", sample_num)
df_search_result

,search_string,start_time,end_time,sentence,video_id
0,alerji testi,224.768,230.912,uygulamadan önce alerji testi muhakkak yapmanı...,Lo7m5rbyQSU
1,alerji testi,860.160,863.880,hadi şimdi alerji testi bakalım der hasta sahi...,dD3A1IseY00
2,alerji testi,718.040,719.740,alerji testi,dD3A1IseY00
3,alerji testi,818.680,821.460,alerji testi şu an kaç euroydu,dD3A1IseY00
4,alerji testi,864.340,869.300,deri testi kediköpeklerde de var deri üzerinde...,dD3A1IseY00
...,...,...,...,...,...
408,video konferans,0.000,207.000,cumhurbaşkanı recep tayyip erdoğan tam kısıtla...,EVLxsmlrHW4
409,video konferans,708.655,713.034,demosundan da fark edeceğiniz üzere yepyeni bi...,goOctWXJFYU
410,video konferans,17.951,22.157,bazı video konferans uygulamaları kullanıcılar...,dnw_xhVxReU
411,video konferans,139.961,144.151,ve böylece yeni imaclerde çok daha iyi bir vid...,goOctWXJFYU


In [31]:
df_word_group_time_loc_result = word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,alerji testi,226.114630,227.292932,uygulamadan önce alerji testi muhakkak yapmanı...,Lo7m5rbyQSU
1,alerji testi,860.715224,861.492537,hadi şimdi alerji testi bakalım der hasta sahi...,dD3A1IseY00
2,alerji testi,718.040000,719.740000,alerji testi,dD3A1IseY00
3,alerji testi,818.680000,819.884667,alerji testi şu an kaç euroydu,dD3A1IseY00
4,alerji testi,868.225333,869.300000,deri testi kediköpeklerde de var deri üzerinde...,dD3A1IseY00
...,...,...,...,...,...
408,video konferans,108.976190,112.079365,cumhurbaşkanı recep tayyip erdoğan tam kısıtla...,EVLxsmlrHW4
409,video konferans,711.227662,712.158200,demosundan da fark edeceğiniz üzere yepyeni bi...,goOctWXJFYU
410,video konferans,18.222355,19.375613,bazı video konferans uygulamaları kullanıcılar...,dnw_xhVxReU
411,video konferans,142.133593,143.012975,ve böylece yeni imaclerde çok daha iyi bir vid...,goOctWXJFYU


In [32]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: (x-time_shift))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: (x+time_shift))
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id
0,alerji testi,225.514630,227.892932,uygulamadan önce alerji testi muhakkak yapmanı...,Lo7m5rbyQSU
1,alerji testi,860.115224,862.092537,hadi şimdi alerji testi bakalım der hasta sahi...,dD3A1IseY00
2,alerji testi,717.440000,720.340000,alerji testi,dD3A1IseY00
3,alerji testi,818.080000,820.484667,alerji testi şu an kaç euroydu,dD3A1IseY00
4,alerji testi,867.625333,869.900000,deri testi kediköpeklerde de var deri üzerinde...,dD3A1IseY00
...,...,...,...,...,...
408,video konferans,108.376190,112.679365,cumhurbaşkanı recep tayyip erdoğan tam kısıtla...,EVLxsmlrHW4
409,video konferans,710.627662,712.758200,demosundan da fark edeceğiniz üzere yepyeni bi...,goOctWXJFYU
410,video konferans,17.622355,19.975613,bazı video konferans uygulamaları kullanıcılar...,dnw_xhVxReU
411,video konferans,141.533593,143.612975,ve böylece yeni imaclerde çok daha iyi bir vid...,goOctWXJFYU


In [33]:
df_word_group_time_loc_result.start_time = df_word_group_time_loc_result.start_time.apply(lambda x: round(x))
df_word_group_time_loc_result.end_time = df_word_group_time_loc_result.end_time.apply(lambda x: round(x))
df_word_group_time_loc_result 

,search_string,start_time,end_time,sentence,video_id
0,alerji testi,226,228,uygulamadan önce alerji testi muhakkak yapmanı...,Lo7m5rbyQSU
1,alerji testi,860,862,hadi şimdi alerji testi bakalım der hasta sahi...,dD3A1IseY00
2,alerji testi,717,720,alerji testi,dD3A1IseY00
3,alerji testi,818,820,alerji testi şu an kaç euroydu,dD3A1IseY00
4,alerji testi,868,870,deri testi kediköpeklerde de var deri üzerinde...,dD3A1IseY00
...,...,...,...,...,...
408,video konferans,108,113,cumhurbaşkanı recep tayyip erdoğan tam kısıtla...,EVLxsmlrHW4
409,video konferans,711,713,demosundan da fark edeceğiniz üzere yepyeni bi...,goOctWXJFYU
410,video konferans,18,20,bazı video konferans uygulamaları kullanıcılar...,dnw_xhVxReU
411,video konferans,142,144,ve böylece yeni imaclerde çok daha iyi bir vid...,goOctWXJFYU


In [34]:
df_word_group_time_loc_result["video_url"] = "https://www.youtube.com/watch?v="+df_word_group_time_loc_result['video_id'].map(str)+"&t="+df_word_group_time_loc_result['start_time'].map(str)+"s"
df_word_group_time_loc_result

,search_string,start_time,end_time,sentence,video_id,video_url
0,alerji testi,226,228,uygulamadan önce alerji testi muhakkak yapmanı...,Lo7m5rbyQSU,https://www.youtube.com/watch?v=Lo7m5rbyQSU&t=...
1,alerji testi,860,862,hadi şimdi alerji testi bakalım der hasta sahi...,dD3A1IseY00,https://www.youtube.com/watch?v=dD3A1IseY00&t=...
2,alerji testi,717,720,alerji testi,dD3A1IseY00,https://www.youtube.com/watch?v=dD3A1IseY00&t=...
3,alerji testi,818,820,alerji testi şu an kaç euroydu,dD3A1IseY00,https://www.youtube.com/watch?v=dD3A1IseY00&t=...
4,alerji testi,868,870,deri testi kediköpeklerde de var deri üzerinde...,dD3A1IseY00,https://www.youtube.com/watch?v=dD3A1IseY00&t=...
...,...,...,...,...,...,...
408,video konferans,108,113,cumhurbaşkanı recep tayyip erdoğan tam kısıtla...,EVLxsmlrHW4,https://www.youtube.com/watch?v=EVLxsmlrHW4&t=...
409,video konferans,711,713,demosundan da fark edeceğiniz üzere yepyeni bi...,goOctWXJFYU,https://www.youtube.com/watch?v=goOctWXJFYU&t=...
410,video konferans,18,20,bazı video konferans uygulamaları kullanıcılar...,dnw_xhVxReU,https://www.youtube.com/watch?v=dnw_xhVxReU&t=18s
411,video konferans,142,144,ve böylece yeni imaclerde çok daha iyi bir vid...,goOctWXJFYU,https://www.youtube.com/watch?v=goOctWXJFYU&t=...


In [35]:
df_word_group_time_loc_result.to_excel(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_{sheets.capitalize()}_\
{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx", index=False) 

#### Copy Move And Delete

In [36]:
output_file = glob.glob(f"{lang_folder.capitalize()}_{lang_pair.capitalize()}_*_{sample_num}_Youtube_{time_shift}s_Timeshift_Result.xlsx")
output_file

['Turkish_Dutch_2 gram hybrid_6_Youtube_0.6s_Timeshift_Result.xlsx']

In [37]:
for y in output_file:
    source = y # source directory
    destination = path
    shutil.copy2(source, destination)

In [38]:
for z in output_file:
    try:
        os.remove(z)
    except:
        pass